# 1: Imports and Mounting Google Drive

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Download NLTK data for tokenization
nltk.download('punkt')

# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2: Loading and Preprocessing Data

In [ ]:
# Define the file path to the dataset in Google Drive
file_path = '/content/drive/My Drive/train-balanced-sarcasm.csv'

# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_path)

# Reduce the dataset size to 1% for faster processing
df = df.sample(frac=0.01, random_state=42)

# Remove rows with NaN values in the 'comment' column
df = df.dropna(subset=['comment'])

# Ensure all comments are strings and handle NaN values
df['comment'] = df['comment'].astype(str)
df['comment'] = df['comment'].apply(lambda x: x if pd.notnull(x) else '')

# Preprocessing function to clean and tokenize text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Join tokens back into a single string
    return ' '.join(tokens)

# Apply preprocessing to the 'comment' column
df['processed_comment'] = df['comment'].apply(preprocess_text)

# TF-IDF vectorization to convert text data into numerical features
tfidf_vectorizer = TfidfVectorizer(max_features=2000)
X_tfidf = tfidf_vectorizer.fit_transform(df['processed_comment']).toarray()

# Encode labels into binary values (0 or 1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['label'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42)


# 3: Reshaping Data and Building the Model

In [ ]:
# Reshape data to fit the input shape required by CNN
max_sequence_length = X_tfidf.shape[1]
X_train_reshaped = X_train.reshape((X_train.shape[0], max_sequence_length, 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], max_sequence_length, 1))

# Build the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(max_sequence_length, 1)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# 4: Training the Model

In [ ]:
# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=2, batch_size=32, validation_data=(X_test_reshaped, y_test))


Epoch 1/2
253/253 [==============================] - 24s 85ms/step - loss: 0.6948 - accuracy: 0.5098 - val_loss: 0.6933 - val_accuracy: 0.4758
Epoch 2/2
253/253 [==============================] - 18s 71ms/step - loss: 0.6929 - accuracy: 0.5126 - val_loss: 0.6934 - val_accuracy: 0.4763


# 5: Evaluating the Model and Generating Classification Report

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Accuracy: {accuracy:.4f}')

# Generate a classification report
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test_reshaped) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


64/64 [==============================] - 2s 29ms/step - loss: 0.6934 - accuracy: 0.4763
Accuracy: 0.4763
64/64 [==============================] - 1s 21ms/step
              precision    recall  f1-score   support

           0       0.48      1.00      0.65       963
           1       0.00      0.00      0.00      1059

    accuracy                           0.48      2022
   macro avg       0.24      0.50      0.32      2022
weighted avg       0.23      0.48      0.31      2022



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
